# UV file basics

Raw data from HERA consists of `*.uv` files, which are a streaming file format common in radio astronomy. Behind the scenes, the file contains a cross-correlated visibility for each antenna pair at a specific time and frequency. There are several libraries for interacting with UV files, such as `aipy` and `pyuvdata`. We will spend most of this tutorial using the functionality built into `capo` to learn more about the raw data.

## Using the Librarian
LINK TO LIBRARIAN HERE... DOWNLOAD SAMPLE FILE (which is already on the laptops)

## Basic file information
`uv` files are actually directories that contain the raw data (`visdata`), largely in binary, and metadata (`flags`, `header`, `history`, and `vartable`). Libraries such as `capo` use this information to parse the files into something that we humans can understand.

# Lesson 1: Visualizing Raw Data
Let's start by seeing what the raw data looks like, to understand what we're working with. The basic unit that we'll plot is a waterfall plot. This is a two-dimensional plot of the visibility (the cross-correlated signal between a pair of antennas) as a function of time and frequency. Let's learn how to extract this information.

## Reading in the File
The first step is to read in the file using our library (`aipy`, in this case). We make use of the `plot_uv.py` script. This script takes as an argument the name of the `.uv` file we want to plot. It also needs to know what pair of antennas we'd like to plot, which is done with the `-a` flag. Let's look at the waterfall for antennas 1 and 4. To do this, run the following command in a terminal. You'll have to run it in the same directory where the `.uv` file we downloaded from the Librarian is located.

```
$ plot_uv.py zen.2456680.29213.xx.uv -a 1_4 -p xx --cmap=viridis
```
Notice how there are several frequency channels that are bright yellow for all times. These are channels with heavy RFI contamination. We'll talk about how to remove this noise in a little bit.

Also note some other useful `plot_uv.py` options:
 
 --drng = colorbar range
 
 --max = colorbar maximum value

 # Activity 1: Waterfall Plots
 
In the previous activity, we plotted a waterfall using the plot_uv.py script. This implicitly handled the complicated business of processing the file and reading in the data. Now it's your turn. Please do the following:
  - Start a new python script
  - Use the following command from `capo` to read in the same datafile we used before
  ```
  t,d,f = capo.miriad.read_files(['zen.2456680.29213.xx.uv'],antstr='1_4',polstr='xx')
  ```
  - `d` is a dictionary containing the two-dimensional visibility data
  - Use matplotlib to plot the data
  - Confirm that the plot is the same as when we used `plot_uv.py`
  - Next, make a similar plot for both a short baseline and long baseline
  - Do this for both the amplitude and phase
  - Question: What are the main differences between the plots for the short baseline and long baseline? Why? 

## Sample Solution

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import capo
import aipy

# read in data file
# define filename
uvfile = '/Users/plaplant/Documents/school/penn/champ/zen.2456680.29213.xx.uv'

# use capo to read in a specific baseline (antenna pair) and polarization
# the return values are t (lsts), d (a dictionary of data), and f (flags)
t, d, f = capo.miriad.read_files([uvfile],antstr='1_4',polstr='xx')

# let's understand the data dictionary a little better
print(d.keys())
print(d[(1,4)].keys())

# d is actually a dictionary of dictionaries indexed first by baseline
#   and then by polarization
# since in our capo call we only included one baseline and polarization,
#   that's all that we have

# bearing in mind that visibilities are, in general, complex, let's plot
#   the real and imaginary values
data = np.abs(d[(1,4)]['xx'])
dreal = data.real
dmax = dreal.max()
dmin = dreal.min()
plt.imshow(dreal,aspect='auto',vmax=dmax,
           vmin=dmin,interpolation='nearest')
plt.colorbar(shrink=0.5)
plt.show()

dimag = data.imag
dmax = dimag.max()
dmin = dimag.min()
plt.imshow(dimag,aspect='auto',vmax=dmax,
          vmin=dmin,interpolation='nearest')
plt.colorbar(shrink=0.5)
plt.show()

# TODO: needs a little more fiddling to look like the plot_uv.py result
# Also plot short and long baseline, both amplitude and phase
# Write small paragraph listing main differences

/Users/carinacheng/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


ImportError: No module named capo

# Lesson 2: Delay Transform

Taking the delay transform of our data along frequency is a very important part of our analysis. Let's see why.

```
$ plot_uv.py zen.2456680.29213.xx.uv -a 1_4 -p xx -d --clean=1e-9
```

# Activity 2: Delay Transform Waterfall Plots

Now it's your turn to code up the delay transform process, using the same script you worked on before. Please add the following to your script:

  - Take the fourier transform of your data along the frequency axis
  - Use matplotlib to plot waterfall plots of this data - does it match what we plotted before?
  - Do this for a short baseline and long baseline - are the horizon limits where you expect?

# Lesson 3: Redundant Calibration

Recall the equation for a visibility of baseline $ij$:

$V(\nu, t) = \int {\rm d\Omega} \, A(\Omega,\nu) \, S(\Omega, \nu) \, \exp \left( 2\pi i \nu \vec{b}_{ij}\cdot\hat{s}/c \right)$

In reality, there are some extra _calibration parameters_ needed outside of the integral that make sure everything in an image is the correct flux and in the correct position:

$V(\nu, t) = g^*_i(\nu, t)g^*_j(\nu, t)\int {\rm d\Omega} \, A(\Omega,\nu) \, S(\Omega, \nu) \, \exp \left( 2\pi i \nu \vec{b}_{ij}\cdot\hat{s}/c \right)$

Where the _g_ parameters are complex numbers that we call _complex gains_. Each _g_ parameter can be expressed as an amplitude and a phase:
    
$g(\nu,t) = A(\nu,t)e^{-i\phi(\nu,t)}$
    
(I'm being careful with my $(\nu,t)$ dependencies because these numbers can potentially change for every frequency-time sample.)
    
This means that to calibrate our data, we need to determine a complex number for every antenna (and dipole arm) for every time and every frequency. This could be difficult to do precisely, but HERA's design makes things easier for us. HERA is _highly redundant_, meaning that for lots of antenna combinations, the same visibility should result (**See Antenna Layout Image and Visibility Equation**).
    
Let's check for ourselves:
    
**TASK 1**: plot two redundant visibilities with **THREE** antennae (baselines _ij_ and _jk_), side-by-side one another. Make 4 panels; absolute value and phase for both visibilities.
    
What's similar about these data? What's different?
    
**TASK 2**: take the ratio between the two amplitude panels, and the two phase panels. Do you get a ratio of 1?
    
... probably not! That's because we need to calibrate them -- and HERA's design let's us do this by redundant calibrating visiblities against each other, as opposed to making an image of the sky. 
    
**TASK 3**: using the ratio plots, determine an amplitude and phase calibration $g_j(\nu,t)$
    
**TASK 4**: Choose a different set of redundant baselines that contain the same antenna _j_ and repeat tasks 1 and 2. 
    
Do you think you'll get the same result for $g_j(\nu,t)$? Why/why not?

In [1]:
# Saul's example of all of the above

# Omnical

- system of equations,
- logcal and lincal,
- degeneracies -- gone from requiring a ton of numbers per sample to just 4
- other?

In [ ]:
# Example heracal/omnical run and some plots